<a href="https://colab.research.google.com/github/naitotomoyuki/YoloLearning_with_ggl_colab/blob/main/yolo12m_4FujiccoOkazu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
import os
import shutil

In [2]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [3]:
!export LC_ALL=C.UTF-8
!export LANG=C.UTF-8
!export LANGUAGE=C.UTF-8

In [4]:
output_dir = "runs_experiment"  # runs_experimentが既に存在している場合削除
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
    print(f"✅ ディレクトリ '{output_dir}' を削除しました。")
else:
    print(f"⚠️ ディレクトリ '{output_dir}' は存在しません。")

from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive')

⚠️ ディレクトリ 'runs_experiment' は存在しません。
Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive


In [5]:
!pip install -U torch torchvision torchaudio
!pip install -U ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 888.1/888.1 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 105.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.8/706.8 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.2/287.2 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [6]:
#!rsync -ah --progress /content/drive/MyDrive/dataset/ /content/dataset/
#超重要！！！！！
#ローカルPCでdatasetをdataset.zipに圧縮してMyDriveへコピーしてください。

# ZIPファイルをGoogle DriveからColabへコピー
!rsync -ah --progress /content/drive/MyDrive/dataset.zip /content/

# ZIPファイルを解凍
!unzip -o /content/dataset.zip -d /content/

# (任意) 解凍後のZIPファイルを削除してディスクを節約
!rm /content/dataset.zip

ストリーミング出力は最後の 5000 行に切り捨てられました。
  inflating: /content/dataset/images/train/e515b1ed-Ch120250413102401077.jpg  
  inflating: /content/dataset/images/train/e5184f73-Ch120241227151245.png  
  inflating: /content/dataset/images/train/e5205758-Ch120250228100519.jpg  
  inflating: /content/dataset/images/train/e53e685a-Ch120241225114531.png  
  inflating: /content/dataset/images/train/e5552193-Ch120250414173024832.jpg  
  inflating: /content/dataset/images/train/e5705e4a-Ch120241227151102.png  
  inflating: /content/dataset/images/train/e59b3ace-Ch120250505094915676.jpg  
  inflating: /content/dataset/images/train/e5a3c487-Ch120241121101414.png  
  inflating: /content/dataset/images/train/e5aaee68-Ch120250203095416.jpg  
  inflating: /content/dataset/images/train/e5c3a271-Ch120241203102912.png  
  inflating: /content/dataset/images/train/e5c9ae7a-Ch120250217102739.jpg  
  inflating: /content/dataset/images/train/e5eee2c8-Ch120250505123334267.jpg  
  inflating: /content/dataset/images/train/e

In [7]:
model_path = "/content/drive/MyDrive/yolo12m.pt"
#model_path = "/content/drive/MyDrive/20250801iFujicco_item_yolo12m.pt" #継続学習の場合　名前は任意


In [8]:
# YAML
import yaml

# YAMLファイルを読み込む
data_yaml = "/content/dataset/data.yaml"
with open(data_yaml, "r") as file:
    data = yaml.safe_load(file)

# 変更したいパス
data["test"] = "./images/test"
data["train"] = "./images/train"
data["val"] = "./images/val"

# YAMLファイルを書き戻す
with open(data_yaml, "w") as file:
    yaml.safe_dump(data, file, default_flow_style=False)

print("data.yaml のパスを更新しました。")

data.yaml のパスを更新しました。


In [19]:
import os
from ultralytics import YOLO

def train_yolo(
    model_path,
    data_yaml,
    epochs=200,
    batch_size=8,          # per-GPU バッチ
    #accumulate=4,          # 勾配蓄積 → 有効バッチ 8*4=32
    img_size=1088,
    output_dir="runs_experiment",
    iou_nms=0.95,          # 検証/推論側の内部NMSを緩く
    mosaic=0.15,
    mixup=0.05,
    copy_paste=0.6,
    close_mosaic=15,
    degrees=0.0, shear=0.0, perspective=0.0, translate=0.05, scale=0.4,
    hsv_h=0.015, hsv_s=0.7, hsv_v=0.4,
    erasing=0.2, fliplr=0.5, flipud=0.0,
    lr0=0.008,             # 有効バッチ32前提で安定寄り
    optimizer="SGD",
    cos_lr=True,
    warmup_epochs=3,
    amp=True,
    workers=0,             # Colab/Windows安定
    project_name="okazu_yolov12m_1280_overlap"
):
    print(f"Loading model from {model_path}")
    model = YOLO(model_path)

    os.makedirs(output_dir, exist_ok=True)
    print(f"Training YOLO12 with {epochs} epochs…")

    model.train(
        data=data_yaml,
        epochs=epochs,
        batch=batch_size,
        #accumulate=accumulate,
        imgsz=img_size,

        # 学習率まわり
        optimizer=optimizer,
        lr0=lr0,
        cos_lr=cos_lr,
        warmup_epochs=warmup_epochs,

        # Aug/正則化（密集・重なり向け）
        mosaic=mosaic,
        mixup=mixup,
        copy_paste=copy_paste,
        close_mosaic=close_mosaic,
        degrees=degrees,
        shear=shear,
        perspective=perspective,
        translate=translate,
        scale=scale,
        hsv_h=hsv_h, hsv_s=hsv_s, hsv_v=hsv_v,
        erasing=erasing,
        fliplr=fliplr,
        flipud=flipud,

        # 検証時のNMS挙動（内部は緩くして外側で締める方針に合わせる）
        iou=iou_nms,

        # 実行系
        amp=amp,
        workers=workers,
        project=output_dir,
        name=project_name,
    )

    print(f"Training complete. Results saved to {output_dir}/{project_name}")

# ===== Colab 用パラメータ =====
model_path = "/content/drive/MyDrive/yolo12m.pt"
data_yaml  = "/content/dataset/data.yaml"

# A100 40GB なら batch=8, accumulate=4（有効32）で余裕
# T4 なら batch=4, accumulate=8（有効32）に下げてOK
train_yolo(
    model_path=model_path,
    data_yaml=data_yaml,
    epochs=300,
    batch_size=8,
    #accumulate=4,
    img_size=1088,
    output_dir="runs_experiment",
    iou_nms=0.95,
    mosaic=0.15,
    mixup=0.05,
    copy_paste=0.6,
    close_mosaic=15,
    degrees=0.0, shear=0.0, perspective=0.0, translate=0.05, scale=0.4,
    hsv_h=0.015, hsv_s=0.7, hsv_v=0.4,
    erasing=0.2, fliplr=0.5, flipud=0.0,
    lr0=0.008,
    optimizer="SGD",
    cos_lr=True,
    warmup_epochs=3,
    amp=True,
    workers=0,
    project_name="okazu_yolov12m_1280_overlap"
)


Loading model from /content/drive/MyDrive/yolo12m.pt
Training YOLO12 with 300 epochs…
Ultralytics 8.3.176 🚀 Python-3.11.13 torch-2.8.0+cu128 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=15, cls=0.5, conf=None, copy_paste=0.6, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/content/dataset/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=300, erasing=0.2, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1152, int8=False, iou=0.95, keras=False, kobj=1.0, line_width=None, lr0=0.008, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.05, mode=train, model=/content/drive/MyDrive/yolo12m.pt, momentum=0.937, mosaic=0.15, multi_scale=False, name=okazu_yolov12m_1280_overlap9,

train: Scanning /content/dataset/labels/train.cache... 2746 images, 3 backgrounds, 0 corrupt: 100%|██████████| 2746/2746 [00:00<?, ?it/s]

train: /content/dataset/images/train/ba79da94-Ch120241225104204.png: 1 duplicate labels removed
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 3593.6±418.5 MB/s, size: 994.3 KB)



val: Scanning /content/dataset/labels/val.cache... 784 images, 1 backgrounds, 0 corrupt: 100%|██████████| 784/784 [00:00<?, ?it/s]

val: /content/dataset/images/val/fc0f6c84-Ch120241121101144.png: 1 duplicate labels removed
Plotting labels to runs_experiment/okazu_yolov12m_1280_overlap9/labels.jpg... 


optimizer: SGD(lr=0.008, momentum=0.937) with parameter groups 123 weight(decay=0.0), 130 weight(decay=0.0005), 129 bias(decay=0.0)
Image sizes 1152 train, 1152 val
Using 0 dataloader workers
Logging results to runs_experiment/okazu_yolov12m_1280_overlap9
Starting training for 300 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/344 [00:00<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 822.00 MiB. GPU 0 has a total capacity of 39.56 GiB of which 438.88 MiB is free. Process 58154 has 39.12 GiB memory in use. Of the allocated memory 36.21 GiB is allocated by PyTorch, and 2.35 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
output_dir = "runs_experiment"
project_name = "okazu_yolov12m_1280_overlap"
best_model_path = f"{output_dir}/{project_name}/weights/best.pt"

if os.path.exists(best_model_path):
    print(f"Downloading: {best_model_path}")
    files.download(best_model_path)
else:
    print(f"[ERROR] best.pt が見つかりません: {best_model_path}")
